# Demonstration notebook to search, list product and download a band

This notebook is defined in four different sections:
1. Import requirements and definition of the parameters
2. Search for cached products
3. Listing the bands of one product
4. Downloading a band

## Import requirements and definition of the parameters

In [1]:
import os
import os.path as path
import requests
import json

Access to the Catalogue API
This API is listing the EO products that have been downloaded within Max-ICS Cache
The cache is populated thanks to user jobs that can be created using the user job api

In [2]:
API_CATALOGUE_BASE_URL = "https://api.earthlab.lu/priv/sat-job-catalog"
API_PRODUCT_BANDS_BASE_URL = "https://api.earthlab.lu/priv/sat-product-job/v1/products/{product_id}/bands"
API_PRODUCT_BAND_BASE_URL = "https://api.earthlab.lu/priv/sat-product-job/v1/products/{product_id}/bands/{file_id}"
API_AUTH_BASE_URL = "https://max-ics.earthlab.lu/authServer/api/v1/connection"

In [3]:
PROVIDER = "sentinel-2"
GEO_AREA = {"type":"Polygon","coordinates":[[[-104.852226585911,39.600325260831596],[-97.9901366137382,39.600325260831596],[-97.9901366137382,43.20496589647098],[-104.852226585911,43.20496589647098],[-104.852226585911,39.600325260831596]]]}
USERNAME = "<user>" ## Please modify
PASSWORD = "<pass>" ## Please modify

Various function to can be used to authenticate, search for products, list bands and download bands

In [4]:
def get_auth_token() -> type:
    """Function to get an authentication Bearer
    :return: JWT token
    :rtype: str"""
    payload = {"user_name": USERNAME,"password": PASSWORD,"with_token": False}
    req_auth = requests.post(API_AUTH_BASE_URL, json=payload, )
    req_auth.raise_for_status()
    bearer = req_auth.headers['Set-Cookie'].split(";")[0]
    return "Bearer " + bearer.split("=")[1]

def search_for_cached_products(geo_area: dict, provider: str, limit: int) -> list:
    """Function to search for cached product within a certain area
    :param geo_area: GeoJSON for the rectangled searched area
    :type geo_area: dict (GeoJSON)
    :param provider: name of the provider (one of: landsat-8, sentinel-2...)
    :type provider: str
    :param limit: Number of result (max 1000)
    :type limit: int
    :return: List of products
    :rtype: list
    """
    ## Filter results within given flattened coordinates as a single array.
    ## It can contains 4 coordinates (bbox) or more than 8 (polygon)
    within = []
    for point in geo_area['coordinates'][0]:
        within.append(point[0])
        within.append(point[1])
    url = API_CATALOGUE_BASE_URL + "/products?provider=" + provider 
    for coord in within:
        url = url + "&within=" + str(coord)
    url = url + "&limit=" + str(limit)
    req = requests.get(url=url, headers={"authorization": get_auth_token()})
    req.raise_for_status()
    return req.json()
    

def download_product_bands(product_id: str) -> list:
    """Function to get the bands information for one particular product
    :param product_id: ID of the product
    :type product_id: str
    :return: list of products
    :rtype: list
    """
    url = API_PRODUCT_BANDS_BASE_URL.format(product_id=product_id)
    req = requests.request(method="get", url=url, headers={"authorization": get_auth_token()})
    req.raise_for_status()
    return req.json()

def download_one_band(product_id: str, file_id: str, filename: str, out_path: str = "./") -> None:
    """Function to download a product from the API
    :param product_id: ID of the product
    :type product_id: str
    :param file_id: ID of the band
    :type file_id: str
    :param filename: Name of the file to save
    :type filename: str
    :param out_path: folder to use for saving the file [Default to: './']
    :type out_path: str
    """
    url = API_PRODUCT_BAND_BASE_URL.format(product_id=product_id, file_id=file_id)
    req = requests.get(url=url, headers={"authorization": get_auth_token()}, stream=True)
    req.raise_for_status()
    handle = open(os.path.join(out_path, filename), "wb")
    for chunk in req.iter_content(chunk_size=8192):
        if chunk:  # filter out keep-alive new chunks
            handle.write(chunk)


# First making the search within the cached products

In [5]:
product_list = search_for_cached_products(geo_area=GEO_AREA, provider=PROVIDER, limit=1000)
print("Got ", len(product_list), " products")

Got  921  products


# Second listing the bands

In [7]:
one_product_id = product_list[0]['id']
one_product_bands = download_product_bands(product_id=one_product_id)
print("The first product has ", len(one_product_bands), " pseudo-bands (files respective to band and resolutions):")
print("------------------------------------|---------------------------------------------------------------")
print("Filename                            |  Band ID")
print("------------------------------------|---------------------------------------------------------------")
for band in one_product_bands:
    print(band['filename'], " | ", band['id'])

The first product has  35  pseudo-bands (files respective to band and resolutions):
------------------------------------|---------------------------------------------------------------
Filename                            |  Band ID
------------------------------------|---------------------------------------------------------------
T14TKL_20200609T172911_B07_60m.jp2  |  UzJBX01TSUwyQV8yMDIwMDYwOVQxNzI5MTFfTjAyMTRfUjA1NV9UMTRUS0xfMjAyMDA2MDlUMjE1MDEyLlNBRkUvVDE0VEtMXzIwMjAwNjA5VDE3MjkxMV9CMDdfNjBtLmpwMg==
T14TKL_20200609T172911_B8A_60m.jp2  |  UzJBX01TSUwyQV8yMDIwMDYwOVQxNzI5MTFfTjAyMTRfUjA1NV9UMTRUS0xfMjAyMDA2MDlUMjE1MDEyLlNBRkUvVDE0VEtMXzIwMjAwNjA5VDE3MjkxMV9COEFfNjBtLmpwMg==
T14TKL_20200609T172911_WVP_60m.jp2  |  UzJBX01TSUwyQV8yMDIwMDYwOVQxNzI5MTFfTjAyMTRfUjA1NV9UMTRUS0xfMjAyMDA2MDlUMjE1MDEyLlNBRkUvVDE0VEtMXzIwMjAwNjA5VDE3MjkxMV9XVlBfNjBtLmpwMg==
T14TKL_20200609T172911_TCI_60m.jp2  |  UzJBX01TSUwyQV8yMDIwMDYwOVQxNzI5MTFfTjAyMTRfUjA1NV9UMTRUS0xfMjAyMDA2MDlUMjE1MDEyLlNBRkUvVDE0VEtMXzIw

Getting more information about the band

In [8]:
## Example of band information:
print("Example of information for ", band['filename'])
print("Name: ", band['details']['name'])
print("Start of wavelength: ", band['details']['start_wavelength_nm'], " nm")
print("End of wavelength: ", band['details']['end_wavelength_nm'], " nm")
print("Resolution: ", band['details']['pixel_size_m'], " m/px")

Example of information for  T14TKL_20200609T172911_B08_10m.jp2
Name:  NIR
Start of wavelength:  784.5  nm
End of wavelength:  899.5  nm
Resolution:  10  m/px


# Downoading the band as file

In [9]:
## Example of code to download the mentioned band
download_one_band(product_id=one_product_id, file_id=band['id'], filename=band['filename'])